# Moving Pre Pre-alginment workflow

I have been refactoring the pre-pre and pre-aln workflows. Now I want to move the output over to its new location. 

In [4]:
# %load ../start.py
# Load useful extensions

# Activate the autoreload extension for easy reloading of external packages
%reload_ext autoreload
%autoreload 2

# Trun on the water mark
%reload_ext watermark
%watermark -u -d -g

# Load ipycache extension
%reload_ext ipycache
from ipycache import CacheMagics
CacheMagics.cachedir = '../cachedir'

# Add project library to path
import sys
sys.path.insert(0, '../../lib/python')

# The usual suspects
import os
import numpy as np
import pandas as pd

# plotting
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
sns.set_context('poster')

# Turn off scientific notation
np.set_printoptions(precision=5, suppress=True)


last updated: 2017-10-23 
Git hash: f4928a44e103faef45c057e2d4a3c312df6f3bfb


In [5]:
# %load ../../bin/load.py
from pymongo import MongoClient
with open('/home/fearjm/Projects/ncbi_remap/output/.mongodb_host', 'r') as fh:
    host = fh.read().strip()
client = MongoClient(host=host, port=27022)
db = client['sra2']
remap = db['remap']


In [18]:
import shutil
from pathlib import Path

In [10]:
store = pd.HDFStore('../../sra.h5', mode='r')

In [69]:
def move_fastq(srx=None, srr=None, fname=None):
    orig1 = '../../output/pre-prealignment/raw/{srx}/{srr}/{srr}_1.fastq.gz'.format(srx=srx, srr=srr)
    orig2 = '../../output/pre-prealignment/raw/{srx}/{srr}/{srr}_2.fastq.gz'.format(srx=srx, srr=srr)
    new1 = '../../output/prealn-wf/samples/{srx}/{srr}/{srr}_1.fastq.gz'.format(srx=srx, srr=srr)
    new2 = '../../output/prealn-wf/samples/{srx}/{srr}/{srr}_2.fastq.gz'.format(srx=srx, srr=srr)
    
    os.makedirs(os.path.dirname(new1), exist_ok=True)
    os.makedirs(os.path.dirname(new2), exist_ok=True)
    
    try:
        if fname == 'SE':
            shutil.move(orig1, new1)
            Path(new2).touch()
        elif fname == 'PE':
            shutil.move(orig1, new1)
            shutil.move(orig2, new2)
        elif fname == 'keep_R1':
            shutil.move(orig1, new1)
            Path(new2).touch()
        elif fname == 'keep_R2':
            Path(new1).touch()
            shutil.move(orig2, new2)
    except FileNotFoundError as e:
        pass

In [71]:
for i, record in store['layout/SE'].iterrows():
    move_fastq(fname='SE', **record.to_dict())

In [72]:
for i, record in store['layout/PE'].iterrows():
    move_fastq(fname='PE', **record.to_dict())

In [74]:
for i, record in store['layout/keep_R1'].iterrows():
    move_fastq(fname='keep_R1', **record.to_dict())

In [73]:
for i, record in store['layout/keep_R2'].iterrows():
    move_fastq(fname='keep_R2', **record.to_dict())